In [ ]:
# imports
from pathlib import Path
import geopandas as gpd
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
import rioxarray as rxr
import seaborn as sns

In [ ]:
data_path = Path("data")

reference_raster = rasterio.open(data_path / "landcover_2010_nyc_3ft_mn.tif")

In [ ]:
# read geoparquet into a gdf
out_data_name = "buildings"
file_ext = "parquet"
bldg_gdf = gpd.read_parquet(data_path / f"{out_data_name}.{file_ext}")

# fill null values in either elevation col with 0s
bldg_gdf.fillna({'groundelev':0,'heightroof':0},inplace=True)

print(f"num of rows in bldg dataset: {len(bldg_gdf)}")

In [ ]:
# calculate integer roof height relative to sealevel, not grade
bldg_gdf["roof_hgt_from_sealvl"] = (bldg_gdf.heightroof + bldg_gdf.groundelev).astype(
    int
)

In [ ]:
bldg_gdf.info()

In [ ]:
# subset geoparquet to only include MN for testing
mn_bldg_gdf = bldg_gdf.query("base_bbl.str.startswith('1')")

print(f"num of rows in mn bldg dataset: {len(mn_bldg_gdf)}")

In [ ]:
count = mn_bldg_gdf.groupby(by=['roof_hgt_from_sealvl']).size()
display(count)

In [ ]:
count.plot()

In [ ]:
## rasterize bldg footprints

geom = [shapes for shapes in mn_bldg_gdf.geometry]

# generate tuples of geometry, value pairs, where value is the attribute value you want to burn
geom_value = (
    (geom, value)
    for geom, value in zip(mn_bldg_gdf.geometry, mn_bldg_gdf["roof_hgt_from_sealvl"])
)

# Rasterize vector using the shape and transform of the raster
rasterized = rasterio.features.rasterize(
    shapes=geom_value,
    out_shape=reference_raster.shape,
    transform=reference_raster.transform,
    all_touched=True,
    fill=0,  # background value
    merge_alg=rasterio.enums.MergeAlg.replace,
    # dtype=np.int16,
)

In [ ]:
out_raster = "mn_bldg_raster.tif"

with rasterio.open(
    fp=data_path / f"{out_raster}",
    mode="w",
    driver="GTiff",
    width=reference_raster.width,   # width in rows
    height=reference_raster.height, # height in rows
    count=1,                        # number of bands
    crs=reference_raster.crs,
    transform=reference_raster.transform,
    dtype=rasterio.float32,           # dtype must be float, else lose elev resolution
) as dst:
    dst.write(
        arr=rasterized,             # array to write to raster
        indexes=1,                  # bands to write to
        )

In [ ]:
mn_bldg_raster = rasterio.open(data_path / "mn_bldg_raster.tif")
rasterio.plot.show(mn_bldg_raster)

In [ ]:
# Prettier plotting with seaborn
sns.set_theme(font_scale=1.5, style="whitegrid")

In [ ]:
# Open data 
mn_bldg_xarray = rxr.open_rasterio(data_path / "mn_bldg_raster.tif", masked=True)

# View object dimensions
mn_bldg_xarray.shape

In [ ]:
# Plot a histogram
bins = [x for x in range(5,2000,5)]
f, ax = plt.subplots(figsize=(10, 6))
mn_bldg_xarray.plot.hist(
    ax=ax,
    color="purple",
    bins=bins,
)
ax.set(title="MN BLDGS", xlabel="Roof Height Above Sealevel (ft)", ylabel="Frequency")

plt.show()